In [ ]:
!pip install fiftyone

In [ ]:
import fiftyone as fo
import os
fo.list_datasets()

In [ ]:
%%bash
# Create the local directory if it doesn't exist
mkdir -p /missouri

# Download files using gsutil
gsutil -m cp -r "gs://public-datasets-lila/missouricameratraps/images/Set1/1.60-Red_Fox/SEQ75195" "/missouri"



In [ ]:
name = 'missouri'
if fo.dataset_exists(name):
  dataset = fo.load_dataset(name)
else:
  dataset = fo.Dataset.from_images_dir("/missouri/")
  dataset.name = name
  dataset.compute_metadata()
  dataset.save()

In [ ]:
fo.list_datasets()

In [ ]:
fo.launch_app(dataset)

In [ ]:
import zipfile
import io
import urllib
import json

url = "https://lila.science/public/lila-md-results/missouri-camera-traps_mdv5a.0.0_results.json.zip"
access_url = urllib.request.urlopen(url)

z = zipfile.ZipFile(io.BytesIO(access_url.read()))
data = json.loads(z.read(z.infolist()[0]).decode())
print(data['detection_categories'])
print(data['images'][:10])

In [ ]:
sample_images = os.listdir('/missouri/SEQ75195')
keep_data = {os.path.basename(dat['file']): dat['detections'] for dat in data['images'] if any(os.path.basename(samp) in dat['file'] for samp in sample_images)}
print(keep_data)

In [ ]:
cat_dict = data['detection_categories']
for sample in dataset.iter_samples(progress = True, autosave = True):
  dat_name = os.path.basename(sample.filepath)
  dets = keep_data[dat_name]
  add_dets = []
  for det in dets:
    add_dets.append(fo.Detection(
        label = cat_dict[det['category']],
        bounding_box = det['bbox'],
        confidence = det['conf']
    ))
  sample['MegaDetector'] = fo.Detections(detections = add_dets)

In [ ]:
fo.launch_app(dataset)


In [ ]:


from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
inpath = '/content/drive/MyDrive/Colab/ground_truth.csv'
df = pd.read_csv(inpath)
field = 'ground_truth'
for sample in dataset.iter_samples(progress = True, autosave = True):
        sub = df.loc[df['Filepath'].apply(lambda x: os.path.basename(x)) == os.path.basename(sample.filepath)] # this will only work if every file in your dataset has a unique name!!!
        dets = []
        for index, row in sub.iterrows():
            bbox = [
                row['XMin'],
                row['YMin'],
                row['Width'],
                row['Height']
            ]
            det = fo.Detection(
                label = row['Label'],
                bounding_box = bbox,
                confidence = row['Confidence']
            )
            dets.append(det)
        sample[field] = fo.Detections(detections = dets)

In [ ]:
fo.launch_app(dataset)

In [ ]:
results = fo.evaluate_detections(
    dataset,
    pred_field = 'MegaDetector',
    gt_field = 'ground_truth',
    classwise = False
)
results.plot_confusion_matrix()

In [ ]:
from fiftyone import ViewField as VF
high_conf_view = dataset.filter_labels('MegaDetector', VF('confidence') > 0.1, only_matches = False)
results = fo.evaluate_detections(
    high_conf_view,
    pred_field = 'MegaDetector',
    gt_field = 'ground_truth',
    classwise = False
)
results.plot_confusion_matrix()

In [ ]:
# code snippet to make a fiftyone dataset directly from MegaDetector outputs
# AND output results to a csv
!pip install fiftyone
import pandas as pd
import fiftyone as fo
import json
from PIL import Image


with open("batch_output.json", 'r') as f:
    batch_output = json.load(f)
model_name = "MegaDet"
encoding_2_class = batch_output["categories"]
dataset_name = 'my_favorite_dataset'
if fo.dataset_exists(dataset_name):
    fo_dataset = fo.load_dataset(dataset_name)
else:
    fo_dataset = fo.Dataset(dataset_name)
results = []
for img_output in batch_output["annotations"]:
    filepath = img_output["img_id"]
    try:
        sample = fo_dataset[filepath] #exception if not in dataset
    except:
        sample = fo.Sample(filepath=filepath)
        fo_dataset.add_sample(sample)
    img = Image.open(filepath)
    img_width, img_height = img.size
    fiftyone_detections = []
    for i in range(len(img_output["bbox"])):
        xmin, ymin, xmax, ymax = img_output["bbox"][i]
        x = xmin / img_width
        y = ymin / img_height
        w = (xmax - xmin) / img_width
        h = (ymax - ymin) / img_height
        det = fo.Detection(
            label=encoding_2_class[str(img_output["category"][i])],
            bounding_box=[x, y, w, h],
            confidence=img_output["confidence"][i],
        )
        fiftyone_detections.append(det)
        results.append([
          img_output['img_id'],
          encoding_2_class[str(img_output['category'][i])],
          img_output['confidence'][i],
          x,
          y,
          w,
          h
      ])
    sample[model_name] = fo.Detections(detections = fiftyone_detections)
    sample.save()
dataframe = pd.DataFrame(
    results,
    columns=[
        'image_id',
        'category',
        'confidence',
        'xmin',
        'ymin',
        'xmax',
        'ymax'
    ]
)
print(dataframe)
dataframe.to_csv("demo_output/batch_output.csv", index = False)
session = fo.launch_app(fo_dataset, remote=False, port=1122)